In [1]:
from multiprocessing.dummy import active_children
import os
from re import I
import sys
import logging
import argparse

import json
import pandas as pd
import numpy as np

from keras import activations
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Lambda, Concatenate, Reshape, Add, Dropout
from keras.layers import MultiHeadAttention
from keras.models import Model 

from keras.utils.vis_utils import plot_model

from myattention import Attention
from layers import Aggregate
from load import load_data, load_cla_data

### Pre process data

In [ ]:
def preprocess(data_path, fname):
    df = pd.read_csv(data_path + fname)
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values(by='Date').reset_index(drop=True)
    df['Close-1'] = df['Close'].shift(1)
    try: 
        df['c_open'] = (df['Original_Open']/df['Close']) - 1
    except:
        df['c_open'] = (df['Open']/df['Close']) - 1
    df['c_high'] = df['High']/df['Close'] - 1
    df['c_low'] = df['Low']/df['Close'] - 1
    df['c_close'] = df['Close']/df['Close-1'] - 1

    df['adj_close-1'] = df['Adj Close'].shift(1)
    df['c_adj_close'] = df['Adj Close']/df['adj_close-1'] -1

    df['adj_close+1'] = df['Adj Close'].shift(-1)
    df['c_adj_close+1'] = df['adj_close+1']/df['Adj Close'] -1

    df['c_5'] = (df['Adj Close'].rolling(5).sum()/(5*df['Adj Close'])) - 1
    df['c_10'] = (df['Adj Close'].rolling(10).sum()/(10*df['Adj Close'])) - 1
    df['c_15'] = (df['Adj Close'].rolling(15).sum()/(15*df['Adj Close'])) - 1
    df['c_20'] = (df['Adj Close'].rolling(20).sum()/(20*df['Adj Close'])) - 1
    df['c_25'] = (df['Adj Close'].rolling(25).sum()/(25*df['Adj Close'])) - 1
    df['c_30'] = (df['Adj Close'].rolling(30).sum()/(30*df['Adj Close'])) - 1
    df['c_label'] = 0
    # df.loc[df['c_adj_close'] > 0.0055, 'c_label'] = 1
    # df.loc[df['c_adj_close'] < -0.005, 'c_label'] = -1

    # df.loc[df['c_adj_close+1'] > 0.0055, 'c_label'] = 1
    # df.loc[df['c_adj_close+1'] < -0.005, 'c_label'] = -1
    df.loc[df['c_adj_close+1'] >= 0, 'c_label'] = 1
    df.loc[df['c_adj_close+1'] < 0, 'c_label'] = -1
    df = df[['c_open', 'c_high', 'c_low', 'c_close','c_adj_close','c_5', 'c_10', 'c_15', 'c_20', 'c_25','c_30', 'c_label', 'Adj Close']]
    # df = df.iloc[:-1,:]
    df.iloc[:29, :] = -123321.000000
    return df

In [ ]:
# index 
processed_df = preprocess('./data/index/raw/', 'SPY.csv')
processed_df.to_csv('./data/index/preprocessed/'+ 'snp500_p.csv', index=False, header=False)

In [ ]:
# stocks
data_path = './data/kdd17/price_long_50/'
fnames = [fname for fname in os.listdir(data_path) if
            os.path.isfile(os.path.join(data_path, fname))]
for fname in fnames:
    processed_df = preprocess(data_path, fname)
    processed_df.to_csv('./data/kdd17/processed/' + fname, index=False, header=False)

### Create the model and test wtih dummy data

In [2]:
params = {
    "feature_dim": 11,
    "ticker_size": 2,
}

time_steps=3
units=11

index_X_train = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],])
stock1 = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],])
stock2 = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],])
stock_X_train_list = [stock1, stock2]
y_train = np.array([[1,0],
                    [1,0],
                    [0,0],
                    [0,1],
                    [1,1]])

test = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]]])


In [3]:
def time_axis_attention(model_input, feature_dim, units):
    x = Dense(feature_dim, activation='tanh')(model_input)
    x = LSTM(units, return_sequences=True)(x)
    x = Attention(feature_dim)(x)
    x = BatchNormalization()(x)
    return x

index_input = Input(shape=(time_steps, params['feature_dim']), name='index_input')
index_model = time_axis_attention(index_input, params['feature_dim'], units)
# index_model = Model(inputs=index_input, outputs=index_model)

stock_input_list = [Input(shape=(time_steps, params['feature_dim'])) for stock_input in range(params['ticker_size'])]

context_aggs = [None]*(params['ticker_size'])
for i in range(params['ticker_size']):
    stock_input = stock_input_list[i]
    stock_model = time_axis_attention(stock_input, params['feature_dim'], units)
    # stock_model = Model(inputs=stock_input, outputs=stock_model)
    context_aggs[i] = Aggregate(units)([index_model, stock_model])

x = context_aggs[0]
for i in range(1, params['ticker_size']):
    x = Concatenate()([x, context_aggs[i]])
x = Reshape((params['ticker_size'], -1))(x)
print(x.shape)

# Data Axis context
mha = MultiHeadAttention(num_heads=2, key_dim=2)(x, x)
# Nonlinear Transformation
addition = Add()([x, mha])
x = Dense(units)(addition)
x = Add()([addition, x])
x = Dropout(rate=0.2)(x)
x = activations.tanh(x)
x = Dense(1, activation='sigmoid')(x)
print(x.shape)
model = Model(inputs=[index_input, *stock_input_list], outputs=x)

model.compile(loss='mae', optimizer='adam')
plot_model(model, to_file='./image/model_plot.png', show_shapes=True, show_layer_names=True)
model.fit(x =[index_X_train, stock_X_train_list], y=y_train ,epochs=5)

2022-07-29 14:58:05.573787: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


(None, 2, 11)
(None, 2, 1)
Epoch 1/5


2022-07-29 14:58:11.777891: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 2600 num_cores: 12 environment { key: "cpu_instruction_set" value: "SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 12582912 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2022-07-29 14:58:11.778081: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 2600 num_cores: 12 environment { key: "cpu_instruction_set" value: "SSE, SSE2,

1/1 [==============================] - 5s 5s/step - loss: 0.5000
Epoch 2/5
1/1 [==============================] - 0s 8ms/step - loss: 0.4999
Epoch 3/5
1/1 [==============================] - 0s 9ms/step - loss: 0.4999
Epoch 4/5
1/1 [==============================] - 0s 8ms/step - loss: 0.4994
Epoch 5/5
1/1 [==============================] - 0s 7ms/step - loss: 0.4999


In [6]:
index_X_test = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]]])

stock1_test = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]]])
stock2_test = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]]])
stock_X_test_list = [stock1_test, stock2_test]
pred = model.predict([index_X_test, stock_X_test_list])
pred

1/1 [==============================] - 0s 21ms/step


array([[[0.4407476 ],
        [0.42292118]]], dtype=float32)